In [1]:
import os
import subprocess
import pandas as pd #pip install pandas
import xlwt #pip install xlwt
from openpyxl import load_workbook #pip install openpyxldd
from tqdm import tqdm
import datetime

**IP-пространство:**  
10.2.124.1-10.2.127.254

In [2]:
snmp_dir = "C:/Users/Никита Широкопетлев/Documents/SnmpGet/"
diap_3 = [str(i) for i in range(124, 128)]
diap_4 = [str(i) for i in range(1, 255)]
for i in diap_3:
    for j in diap_4:
        IP = '10.2.' + i + '.' + j
        try:
            response = subprocess.check_output("ping " + IP)
            #print(response.decode("cp866"))
            print('IP:', IP, 'доступен')
            res = subprocess.check_output(snmp_dir + "SnmpGet.exe -r:" + IP + 
                    ' -o:' + '1.3.6.1.2.1.43.5.1.1.16.1').decode("cp866").split('\n')
                
            print(str(res[-2].split('=')[-1]).strip('\r'))
        except:
            print('IP:', IP, 'не доступен')
        

IP: 10.2.124.1 доступен
%Failed to get value of SNMP variable. Timeout.
IP: 10.2.124.2 не доступен
IP: 10.2.124.3 доступен
%Failed to get value of SNMP variable. Timeout.
IP: 10.2.124.4 доступен
HP LaserJet MFP M436
IP: 10.2.124.5 доступен
%Failed to get value of SNMP variable. Timeout.
IP: 10.2.124.6 доступен
%Failed to get value of SNMP variable. Timeout.
IP: 10.2.124.7 доступен
%Failed to get value of SNMP variable. Timeout.
IP: 10.2.124.8 доступен
%Failed to get value of SNMP variable. Timeout.
IP: 10.2.124.9 доступен
%Failed to get value of SNMP variable. Timeout.
IP: 10.2.124.10 доступен
%Failed to get value of SNMP variable. Timeout.
IP: 10.2.124.11 доступен
%Failed to get value of SNMP variable. Timeout.
IP: 10.2.124.12 доступен
%Failed to get value of SNMP variable. Timeout.
IP: 10.2.124.13 доступен
%Failed to get value of SNMP variable. Timeout.
IP: 10.2.124.14 доступен
%Failed to get value of SNMP variable. Timeout.
IP: 10.2.124.15 доступен
SEC30CDA737D7DA
IP: 10.2.124.16 до

In [3]:
snmp_dir = "C:/Users/Никита Широкопетлев/Documents/SnmpGet/"
def get_IOD(wb, ser):
    IOD = {}
    work_sheet_IOD = wb['IOD']
    flag = False
    counter = 0
    for rows in work_sheet_IOD.iter_rows(values_only=True):
        if rows[2] == ser[:2] and flag == False:
            #print('Модель:', rows[1])
            flag = True
        elif flag == True and counter < 9:
            if rows[0] and rows[0] != 'Модель':
                IOD[rows[0]] = rows[1]
            counter += 1
    return IOD

def search_cell(name_cell, work_sheet):
    for cell in range(1, 100):
        if work_sheet.cell(row = 1, column = cell).value == name_cell:
            break
    print('Столбец "', name_cell, '": ', cell, sep = '')
    return cell

In [5]:
wb = load_workbook('./printers_v2.xlsx')
print(wb.sheetnames)
work_sheet = wb['MAIN']
cell_ser = search_cell('Серийный номер', work_sheet) - 1
cell_IP = search_cell('IP', work_sheet) - 1

printer = 'AL75023437'

['MAIN', 'IOD', 'AL75023437', 'CNB1L7J286', 'E75198L1N182781', 'E75373J7N561817', 'E75373J7N561836', 'VCF7538360', 'VCG8887941', 'VCG8Y01076', 'VCG9118770', 'ZDP1BJAF80005JJ', 'ZDP1BJFF9000RLT', 'ZDP1BJFF9000TJP', 'ZDP1BJFF9000VXY']
Столбец "Серийный номер": 2
Столбец "IP": 3


In [6]:
start_row = 2
for row in tqdm(work_sheet.iter_rows(min_row=2, values_only=True)):
    if row[cell_ser] == printer:
        try:
            IOD = get_IOD(wb, ser=row[cell_ser])
            #print(IOD)
            for key, value in IOD.items():
                res = subprocess.check_output(snmp_dir + "SnmpGet.exe -r:" + str(row[cell_IP]) + 
                                      ' -o:' + value).decode("cp866").split('\n')
                
                IOD[key] = str(res[-2].split('=')[-1]).strip('\r')
            #print(IOD)

            work_sheet_pr = wb[IOD['Серийный номер принтера']]
            cur_row = 1
            flag = False
            for row_pr in work_sheet_pr.iter_rows(values_only=True):
                if row_pr[0] in IOD:
                    try:
                        work_sheet_pr.cell(row=cur_row, column=2).value = int(IOD[row_pr[0]])
                    except:
                        work_sheet_pr.cell(row=cur_row, column=2).value = IOD[row_pr[0]]
            
                if row_pr[0] == 'Дата проверки':
                    work_sheet_pr.cell(row=cur_row, column=2).value = datetime.datetime.now()

                if flag and not row_pr[0]:
                    work_sheet_pr.cell(row=cur_row, column=1).value = str(datetime.datetime.now().strftime("%d.%m.%y %H:%M"))
                    work_sheet_pr.cell(row=cur_row, column=2).value = int(IOD['Оставшееся число копий тонера'])/int(IOD['Максимальное число копий тонера'])
                    work_sheet_pr.cell(row=cur_row, column=3).value = int(IOD['Кол-во напечатанных страниц'])
                    flag = False

                if row_pr[0] == 'Дата':
                    flag = True
    
                cur_row += 1
        except:
            print(IOD)
            print("Printer", row[1], 'failed')


13it [00:00, 81.50it/s]


In [7]:
wb.save('./printer_test.xlsx')

In [15]:
print(datetime.datetime.now().strftime("%d.%m.%y %H:%M"))

01.08.22 17:26
